<h2>Team Members</h2>
<ul>
<li>Sohad Hossam Eldin 1190019</li>
<li>Bassant Hisham Mohamed 1190018</li>
<li>Yasmin Hashem Niazy 4200013</li>
<li>Mary Ashraf 1190322</li>
</ul>

<h4>Imports</h4>

In [11]:
from imports import *

<h3>Dataset class<h3>

In [12]:
class ArabicDataset(Dataset):

    def __init__(self, path):

        dataset_txt = open(path, "r", encoding='utf-8').read()
        #print(type(dataset_txt))

        chars_to_remove = r"\(\s*[a-zA-Z0-9_-]+\s*/\s*[a-zA-Z0-9_-]+\s*\)|[a-zA-Z0-9_-]+" 
        dataset_cleaned = re.sub(chars_to_remove, "", dataset_txt)
        dataset_sentences = re.split(r"\s*\.\s*|\n|\s*،\s*|\s*:\s*|\s*[()]\s*|\s*؛\s*|\s*؟\s*|\s*!\s*|\s*\"\s*",dataset_cleaned)

        self.x_train_letters,y_train_letters,self.x_train_words,y_train_words= self.Tokenizing(dataset_sentences)

        self.y_train_letters_numbered = LabelEncoder().fit_transform(y_train_letters)
        self.y_train_words_numbered = LabelEncoder().fit_transform(y_train_words)

        #dictionary of the labels(words,letters) before converted to numbers
        self.encoding_mapping_y_letters = {encoded_label: original_label for encoded_label, original_label  in zip(self.y_train_letters_numbered, y_train_letters)}
        self.encoding_mapping_y_words = {encoded_label: original_label for encoded_label, original_label  in zip( self.y_train_words_numbered, y_train_words)}  

        # print(self.y_train_letters_numbered.shape)
        # print(self.y_train_words_numbered.shape)
        # print( self.y_train_letters_numbered[:10])
        # print( self.y_train_words_numbered[:10])

        self.y_train_letters_numbered = self.y_train_letters_numbered.reshape((len(self.y_train_letters_numbered), 1))
        self.y_train_words_numbered = self.y_train_words_numbered.reshape((len(self.y_train_words_numbered), 1))

        # print(self.y_train_letters_numbered.shape)
        # print(self.y_train_words_numbered.shape)
        # print( self.y_train_letters_numbered[:10])
        # print( self.y_train_words_numbered[:10])


    def Tokenizing(self,dataset_sentences):

        x_train_letters = []
        y_train_letters = []
        x_train_words = []
        y_train_words = []
        for sentence in dataset_sentences:
            sentence_wihout_diacratics = strip_tashkeel(sentence)
            tokens = word_tokenize(sentence, language="arabic", preserve_line=True)
            tokens_wihtout_diacratics = word_tokenize(sentence_wihout_diacratics, language="arabic", preserve_line=True)

            for word in tokens:
                text, inputs, diacritics =util.extract_haraqat(word)
                # inputs.insert(0, "<s>")
                # inputs.append("</s>")

                diacritics.insert(0, "<s>")
                diacritics.append("</s>")
                
                x_train_letters.append(inputs)
                y_train_letters.extend(diacritics)


            if(len(tokens)) :
                tokens.insert(0, "<s>")
                tokens.append("</s>")
                y_train_words.extend(tokens)
            if(len(tokens_wihtout_diacratics)) :
                tokens_wihtout_diacratics.insert(0, "<s>")
                tokens_wihtout_diacratics.append("</s>")
                x_train_words.append(tokens_wihtout_diacratics)

        
        # print(x_train_letters[0:10])
        # print(y_train_letters[0:10])
        # print(y_train_words[0:10])
        # print(x_train_words[0:10])
        return x_train_letters,y_train_letters,x_train_words,y_train_words

    def __len__(self):
        return len(self.x_train_letters), len(self.x_train_words)

    # get a row at an index
    # The __getitem__ function loads and returns a sample from the dataset at the given index idx
    def __getitem__(self, idx):
        return self.x_train_letters[idx], self.y_train_letters_numbered[idx], self.x_train_words[idx], self.y_train_words_numbered[idx]

    # get indexes for train and test rows
    # def get_splits(self, n_test=0.33):
    #     # determine sizes
    #     test_size = round(n_test * len(self.X))
    #     train_size = len(self.X) - test_size
    #     # calculate the split
    #     return random_split(self, [train_size, test_size])

<h4>Reading the dataset</h4>

In [13]:
# dataset_txt = open(r"train.txt", "r", encoding='utf-8').read()
# print(type(dataset_txt))
#print(dataset_txt)

<h4>Cleaning the dataset</h4>

In [14]:
# do we need to remove [ :ص]
# chars_to_remove = r"\(\s*[a-zA-Z0-9_-]+\s*/\s*[a-zA-Z0-9_-]+\s*\)|[a-zA-Z0-9_-]+" 
# dataset_cleaned = re.sub(chars_to_remove, "", dataset_txt)
#print(dataset_cleaned)

<h4>Segmenting dataset into sentences</h4>

In [15]:
# r'(\([^)]*\))' should we remove the brackets or not
# dataset_sentences = re.split(r"\s*\.\s*|\n|\s*،\s*|\s*:\s*|\s*[()]\s*|\s*؛\s*|\s*؟\s*|\s*!\s*|\s*\"\s*",dataset_cleaned)
# print(type(dataset_sentences))
# print(dataset_sentences[0:30])

<h4>Tokenizing the sentence</h4>

In [16]:
# x_train_letters = []
# y_train_letters = []
# x_train_words = []
# y_train_words = []
# for sentence in dataset_sentences:
#     sentence_wihout_diacratics = strip_tashkeel(sentence)
#     tokens = word_tokenize(sentence, language="arabic", preserve_line=True)
#     tokens_wihtout_diacratics = word_tokenize(sentence_wihout_diacratics, language="arabic", preserve_line=True)
   
#     for word in tokens:
#         text, inputs, diacritics =util.extract_haraqat(word)
#         x_train_letters.append(inputs)
#         y_train_letters.append(diacritics)

#     if(len(tokens)) :
#         tokens.insert(0, "<s>")
#         tokens.append("</s>")
#         y_train_words.append(tokens)
#     if(len(tokens_wihtout_diacratics)) :
#         tokens_wihtout_diacratics.insert(0, "<s>")
#         tokens_wihtout_diacratics.append("</s>")
#         x_train_words.append(tokens_wihtout_diacratics)

## tokenizing the words

In [17]:
# print(x_train_letters[0:10])
# print(y_train_letters[0:10])
# print(y_train_words[0:10])
# print(x_train_words[0:10])

In [18]:
# print(type(x_train_letters))
# print(x_train_letters[2])

#ouh lala , tokinze is dividing the tashkelat as indpendent letters as well , what does that tell us tho 

In [19]:
# temp = ArabicDataset(r"train.txt") 

# Word2vec Feature Extraction

In [20]:
dataset = ArabicDataset(r"train.txt")

print(dataset.y_train_letters_numbered[:10])
print(dataset.y_train_words_numbered[:10])
print(dataset.x_train_words[:10])
print(dataset.x_train_letters[:10])


# word2vec_model = Word2Vec(
#     x_train_words,  # list of sentences, if you don't have all the data in RAM, you can give a file name to corpus_file
#     vector_size=50,  # output size of word embedding
#     window=4,  # window size
#     min_count=1,  # ignores all the words with total frequency lower than this
#     workers=5,  # number of workers to use
#     sg=1,  # skip-gram
#     hs=0,  # 1 --> hierarchical, 0 --> negative sampling
#     negative=5,  # how many negative samples
#     alpha=0.03,  # the initial learning rate
#     min_alpha=0.0001,  # learning rate will linearly drop to min_alpha as training progresses
#     seed=54,  # random seed
#     iter=10,
#     compute_loss=True,  # number of iterations
# )
model1 = Word2Vec(dataset.x_train_words, min_count = 1, vector_size = 100, window = 5)


[[ 2]
 [ 6]
 [16]
 [ 7]
 [ 7]
 [ 1]
 [ 2]
 [ 6]
 [16]
 [ 1]]
[[     6]
 [ 91505]
 [     5]
 [     6]
 [  6306]
 [ 91125]
 [ 21191]
 [154522]
 [  9139]
 [     5]]
[['<s>', 'قوله', '</s>'], ['<s>', 'أو', 'قطع', 'الأول', 'يده', 'إلخ', '</s>'], ['<s>', 'قال', 'الزركشي', '</s>'], ['<s>', 'ابن', 'عرفة', '</s>'], ['<s>', 'قوله', '</s>'], ['<s>', 'بلفظ', 'يقتضيه', 'كإنكار', 'غير', 'حديث', 'بالإسلام', 'وجوب', 'ما', 'علم', 'وجوبه', 'من', 'الدين', 'ضرورة', '</s>'], ['<s>', 'كإلقاء', 'مصحف', 'بقذر', 'وشد', 'زنار', '</s>'], ['<s>', 'ابن', 'عرفة', '</s>'], ['<s>', 'قول', 'ابن', 'شاس', '</s>'], ['<s>', 'أو', 'بفعل', 'يتضمنه', 'هو', 'كلبس', 'الزنار', 'وإلقاء', 'المصحف', 'في', 'صريح', 'النجاسة', 'والسجود', 'للصنم', 'ونحو', 'ذلك', '</s>']]
[['ق', 'و', 'ل', 'ه'], ['أ', 'و'], ['ق', 'ط', 'ع'], ['ا', 'ل', 'أ', 'و', 'ل'], ['ي', 'د', 'ه'], ['إ', 'ل', 'خ'], ['ق', 'ا', 'ل'], ['ا', 'ل', 'ز', 'ر', 'ك', 'ش', 'ي'], ['ا', 'ب', 'ن'], ['ع', 'ر', 'ف', 'ة']]
